In [1]:
from torch_snippets import * 
from torchvision.datasets import MNIST 
from torchvision import transforms 
device = 'cuda' if torch.cuda.is_available () else 'cpu'
img_transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize([0.5],[0.5]), 
    transforms.Lambda(lambda x: x.to(device))
])

trn_ds = MNIST('../public/data/', transform=img_transform, train=True, download=True)
val_ds = MNIST('../public/data/', transform=img_transform, train=False, download=True)

batch_size = 128
trn_dl = DataLoader(trn_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=False)


/home/delameta/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: module 'torch' has no attribute 'is_available'